In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import hamming_loss, accuracy_score
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
import pandas as pd

In [2]:
mfcc_df = pd.read_csv("../data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv")
mfcc_df

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7190,1.0,-0.554504,-0.337717,0.035533,0.034511,0.443451,0.093889,-0.100753,0.037087,0.081075,...,0.069430,0.071001,0.021591,0.052449,-0.021860,-0.079860,Hylidae,Scinax,ScinaxRuber,60
7191,1.0,-0.517273,-0.370574,0.030673,0.068097,0.402890,0.096628,-0.116460,0.063727,0.089034,...,0.061127,0.068978,0.017745,0.046461,-0.015418,-0.101892,Hylidae,Scinax,ScinaxRuber,60
7192,1.0,-0.582557,-0.343237,0.029468,0.064179,0.385596,0.114905,-0.103317,0.070370,0.081317,...,0.082474,0.077771,-0.009688,0.027834,-0.000531,-0.080425,Hylidae,Scinax,ScinaxRuber,60
7193,1.0,-0.519497,-0.307553,-0.004922,0.072865,0.377131,0.086866,-0.115799,0.056979,0.089316,...,0.051796,0.069073,0.017963,0.041803,-0.027911,-0.096895,Hylidae,Scinax,ScinaxRuber,60


In [3]:
train_df, test_df = train_test_split(mfcc_df, train_size=0.7, random_state=24)
train_df_x = train_df.iloc[:, :-5]
test_df_x = test_df.iloc[:, :-5]
family_train_df = train_df["Family"]
family_test_df = test_df["Family"]
genus_train_df = train_df["Genus"]
genus_test_df = test_df["Genus"]
species_train_df = train_df["Species"]
species_test_df = test_df["Species"]

In [28]:
hamming_results = []
def gaussianSVM(label_train_df, label_test_df):
    parameters = {'estimator__C': np.logspace(-3, 3, 7), 'estimator__gamma': np.logspace(-4, 4, 9)}
    model = OneVsRestClassifier(SVC(kernel='rbf'))
    clf = GridSearchCV(model, parameters, cv=10, scoring='accuracy',n_jobs=-1)
    clf.fit(train_df_x, label_train_df)
    predictions = clf.predict(test_df_x)
    cv_results = clf.cv_results_
    loss = hamming_loss(label_test_df, predictions)
    score = 1-loss
    hamming_results.append(score)
    
    result_df = pd.DataFrame(cv_results["params"])
    result_df["mean_test_score"] = cv_results["mean_test_score"]
    
    pd.set_option('display.max_rows', None) 
    display(result_df)
    print("Best Parameters:", clf.best_params_)
    print("Best Test Score:", clf.best_score_)
    print("Hamming Loss",loss)
    
    return predictions

In [29]:
all_predictions = []
all_true_labels = []
for train_df, test_df in [(family_train_df, family_test_df), (genus_train_df, genus_test_df), (species_train_df, species_test_df)]:
    predictions = gaussianSVM(train_df, test_df)
    all_predictions.append(predictions)
    all_true_labels.append(test_df)

,estimator__C,estimator__gamma,mean_test_score
0,0.001,0.0001,0.606434
1,0.001,0.0010,0.606434
2,0.001,0.0100,0.606434
3,0.001,0.1000,0.606434
4,0.001,1.0000,0.606434
5,0.001,10.0000,0.606434
6,0.001,100.0000,0.606434
7,0.001,1000.0000,0.606434
8,0.001,10000.0000,0.606434
9,0.010,0.0001,0.606434


Best Parameters: {'estimator__C': 10.0, 'estimator__gamma': 1.0}
Best Test Score: 0.9918575846508253
Hamming Loss 0.009263547938860583


,estimator__C,estimator__gamma,mean_test_score
0,0.001,0.0001,0.568110
1,0.001,0.0010,0.568110
2,0.001,0.0100,0.568110
3,0.001,0.1000,0.568110
4,0.001,1.0000,0.568110
5,0.001,10.0000,0.568110
6,0.001,100.0000,0.568110
7,0.001,1000.0000,0.568110
8,0.001,10000.0000,0.568110
9,0.010,0.0001,0.568110


Best Parameters: {'estimator__C': 10.0, 'estimator__gamma': 1.0}
Best Test Score: 0.9908663100760517
Hamming Loss 0.0111162575266327


,estimator__C,estimator__gamma,mean_test_score
0,0.001,0.0001,0.537724
1,0.001,0.0010,0.537923
2,0.001,0.0100,0.539315
3,0.001,0.1000,0.536931
4,0.001,1.0000,0.560957
5,0.001,10.0000,0.656667
6,0.001,100.0000,0.587567
7,0.001,1000.0000,0.565126
8,0.001,10000.0000,0.167394
9,0.010,0.0001,0.537526


Best Parameters: {'estimator__C': 10.0, 'estimator__gamma': 1.0}
Best Test Score: 0.990668291836284
Hamming Loss 0.01157943492357573


In [30]:
import numpy as np
average_hamming = np.mean(hamming_results)
print(average_hamming)

0.9893469198703104


In [ ]:
from sklearn.svm import LinearSVC
def l1penalizedSVM(label_train_df_y, label_test_df_y):
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(train_df_x)
    x_test_scaled = scaler.transform(test_df_x)

    parameters = {'estimator__C': np.logspace(1, 6, 15)}
    model = OneVsRestClassifier(LinearSVC(penalty='l1', dual=False, random_state=1,max_iter=30000))
    clf = GridSearchCV(model, parameters, cv=10, scoring='accuracy')
    clf.fit(x_train_scaled, label_train_df_y)
    predictions = clf.predict(x_test_scaled)
    cv_results = clf.cv_results_
    loss = hamming_loss(label_test_df_y, predictions)
    
    result_df = pd.DataFrame(cv_results["params"])
    result_df["mean_test_score"] = cv_results["mean_test_score"]
    
    pd.set_option('display.max_rows', None) 
    display(result_df)
    print("Best Parameters:", clf.best_params_)
    print("Best Score:", clf.best_score_)
    print(f"Hamming Loss: {loss}")

In [ ]:
l1penalizedSVM(family_train_df, family_test_df)

In [ ]:
l1penalizedSVM(genus_train_df, genus_test_df)

In [ ]:
l1penalizedSVM(species_train_df, species_test_df)

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
def l1penalizedSVMImbalance(label_train_df_y, label_test_df_y):
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(train_df_x)
    x_test_scaled = scaler.transform(test_df_x)

    parameters = {'onevsrestclassifier__estimator__C': np.logspace(1, 6, 15)}
    model = OneVsRestClassifier(LinearSVC(penalty='l1', dual=False, random_state=1,max_iter=20000))
    smote_pipeline = make_pipeline(SMOTE(random_state=42), model)
    clf_with_smote = GridSearchCV(smote_pipeline, parameters, cv=10, scoring='accuracy',n_jobs=-1)
    clf_with_smote.fit(x_train_scaled, label_train_df_y)
    predictions_smote = clf_with_smote.predict(x_test_scaled)
    loss_smote = hamming_loss(label_test_df_y, predictions_smote)
    print("With SMOTE:")
    print("Best Parameters:", clf_with_smote.best_params_)
    print("Best Score:", clf_with_smote.best_score_)
    print(f"Hamming Loss: {loss_smote}")

In [ ]:
l1penalizedSVMImbalance(family_train_df, family_test_df)

In [ ]:
l1penalizedSVMImbalance(genus_train_df, genus_test_df)

In [ ]:
l1penalizedSVMImbalance(species_train_df, species_test_df)

In [ ]:


hamming_distance = np.sum(cluster_labels != true_labels) / len(cluster_labels)
hamming_score = np.sum(cluster_labels == true_labels) / len(cluster_labels)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, hamming_loss
from scipy.stats import mode
from collections import Counter

mfcc_df = pd.read_csv("../data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv")

k_values = range(2, 51)  # Starting from 2 because silhouette score can't be computed with k=1

# List to store the silhouette scores for each k
silhouette_scores = []
for num in range(51):
    for k in k_values:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(features_mfcc_df)
        clusters = kmeans.predict(features_mfcc_df)
        silhouette_avg = silhouette_score(features_mfcc_df, clusters)
        silhouette_scores.append(silhouette_avg)
    
    # Find the k with the highest silhouette score
    best_k = k_values[np.argmax(silhouette_scores)]
    
    optimal_k = best_k  # Placeholder value, replace with the actual optimal k found
    features_mfcc_df = mfcc_df.iloc[:, :-5]
    # Perform K-means clustering with the optimal k
    kmeans = KMeans(n_clusters=optimal_k, random_state=42)
    clusters = kmeans.fit_predict(features_mfcc_df)
    
    # Add the cluster labels to the dataframe
    mfcc_df['Cluster'] = clusters
    
    # Initialize dictionaries to store the majority labels for each cluster
    majority_family = {}
    majority_genus = {}
    majority_species = {}
    
    # Determine the majority label for each cluster
    for cluster in range(optimal_k):
        cluster_subset = mfcc_df[mfcc_df['Cluster'] == cluster]
        
        # Count occurrences and find the majority for family, genus, and species
        family_counts = Counter(cluster_subset['Family'])
        genus_counts = Counter(cluster_subset['Genus'])
        species_counts = Counter(cluster_subset['Species'])
        
        majority_family[cluster] = family_counts.most_common(1)[0][0]
        majority_genus[cluster] = genus_counts.most_common(1)[0][0]
        majority_species[cluster] = species_counts.most_common(1)[0][0]
        

        print(f"Cluster {cluster}:")
        print(f"    Majority Family: {majority_family[cluster]}")
        print(f"    Majority Genus: {majority_genus[cluster]}")
        print(f"    Majority Species: {majority_species[cluster]}\n")

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, hamming_loss
from scipy.stats import mode
from collections import Counter
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import hamming_loss, accuracy_score
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

mfcc_df_original = pd.read_csv("../data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv")
k_values = range(2, 51)

# Repeat the procedure 50 times
for repetition in range(50):
    features_mfcc_df = mfcc_df_original.iloc[:, :-5]  # Excluding the last four columns for features
    silhouette_scores = []

    # Compute silhouette scores for each k
    for k in k_values:
        kmeans = KMeans(n_clusters=k,random_state=1)
        clusters = kmeans.fit_predict(features_mfcc_df)
        silhouette_avg = silhouette_score(features_mfcc_df, clusters)
        silhouette_scores.append(silhouette_avg)

    # Find the k with the highest silhouette score
    best_k = k_values[np.argmax(silhouette_scores)]
    print(f"Iteration {repetition + 1}: Best k based on silhouette score is {best_k}")

    # Perform K-means clustering with the optimal k
    kmeans = KMeans(n_clusters=best_k)
    clusters = kmeans.fit_predict(features_mfcc_df)
    mfcc_df = mfcc_df_original.copy()  # Create a fresh copy for this iteration
    mfcc_df['Cluster'] = clusters

    # Initialize dictionaries to store the majority labels for each cluster
    majority_family = {}
    majority_genus = {}
    majority_species = {}

    # Determine the majority label for each cluster
    for cluster in range(best_k):
        cluster_subset = mfcc_df[mfcc_df['Cluster'] == cluster]
        
        family_counts = Counter(cluster_subset['Family'])
        genus_counts = Counter(cluster_subset['Genus'])
        species_counts = Counter(cluster_subset['Species'])
        
        majority_family[cluster] = family_counts.most_common(1)[0][0]
        majority_genus[cluster] = genus_counts.most_common(1)[0][0]
        majority_species[cluster] = species_counts.most_common(1)[0][0]
        
        print(f"Cluster {cluster}:")
        print(f"    Majority Family: {majority_family[cluster]}")
        print(f"    Majority Genus: {majority_genus[cluster]}")
        print(f"    Majority Species: {majority_species[cluster]}\n")

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, hamming_loss
from scipy.stats import mode
from collections import Counter
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import hamming_loss, accuracy_score
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

mfcc_df_original = pd.read_csv("../data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv")
k_values = range(2, 51)

# Repeat the procedure 50 times
for repetition in range(50):
    features_mfcc_df = mfcc_df_original.iloc[:, :-5]  # Excluding the last four columns for features
    silhouette_scores = []

    # Compute silhouette scores for each k
    for k in k_values:
        kmeans = KMeans(n_clusters=k)
        clusters = kmeans.fit_predict(features_mfcc_df)
        silhouette_avg = silhouette_score(features_mfcc_df, clusters)
        silhouette_scores.append(silhouette_avg)

    # Find the k with the highest silhouette score
    best_k = k_values[np.argmax(silhouette_scores)]
    print(best_k)